In [1]:
import time
import warnings
import sys
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectPercentile, f_classif, SelectFromModel
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_recall_curve, auc, mean_squared_error, \
    r2_score, mean_absolute_error,cohen_kappa_score,accuracy_score,f1_score,matthews_corrcoef,precision_score,recall_score
from sklearn.model_selection import train_test_split
import multiprocessing
from xgboost import XGBRegressor, XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesRegressor

start = time.time()
warnings.filterwarnings("ignore")

def standardize(col):
    return (col - np.mean(col)) / np.std(col)

# the metrics for classification
def statistical(y_true, y_pred, y_pro):
    c_mat = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = list(c_mat.flatten())
    se = tp / (tp + fn)
    sp = tn / (tn + fp)
    auc_prc = auc(precision_recall_curve(y_true, y_pro, pos_label=1)[1],
                  precision_recall_curve(y_true, y_pro, pos_label=1)[0])
    acc = (tp + tn) / (tn + fp + fn + tp)
#     acc_skl = accuracy_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pro)
    recall = se
#     recall_skl = recall_score(y_true, y_pred)
    precision = tp / (tp + fp)
#     precision_skl = precision_score(y_true, y_pred)
    f1 = 2 * (precision * recall) / (precision + recall) # F1 = 2 * (precision * recall) / (precision + recall)
#     f1_skl = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true,y_pred)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn) + 1e-8)
#     mcc_skl = matthews_corrcoef(y_true,y_pred)
    return tn,fp,fn,tp,se,sp,auc_prc,acc,auc_roc,recall,precision,f1,kappa,mcc

def all_one_zeros(data):
    if (len(np.unique(data)) == 2):
        flag = False
    else:
        flag = True
    return flag


feature_selection = False
tasks_dic = {'1-AR-Alva-6108-slim-Normalize-group.csv': ['activity']}

In [2]:
# {'actual_estimator__n_estimators': IntUniformDistribution(high=300, low=10, step=1),
#  'actual_estimator__max_depth': IntUniformDistribution(high=11, low=1, step=1),
#  'actual_estimator__min_samples_split': IntUniformDistribution(high=10, low=2, step=1),
#  'actual_estimator__min_samples_leaf': IntUniformDistribution(high=5, low=1, step=1),
#  'actual_estimator__max_features': UniformDistribution(high=1.0, low=0.4),
#  'actual_estimator__min_impurity_decrease': LogUniformDistribution(high=0.5, low=1e-09),
#  'actual_estimator__criterion': CategoricalDistribution(choices=('gini', 'entropy')),
#  'actual_estimator__bootstrap': CategoricalDistribution(choices=(True, False)),
#  'actual_estimator__class_weight': CategoricalDistribution(choices=('balanced', 'balanced_subsample', {}))}

In [3]:
file_name = '1-AR-Alva-6108-slim-Normalize-group.csv'
task_type = 'cla'  # 'reg' or 'cla'
dataset_label = file_name.split('/')[-1].split('_')[0]
tasks = tasks_dic[dataset_label]
OPT_ITERS = 50
repetitions = 10
num_pools = 10
unbalance = True
patience = 100
ecfp = True
# preset the hyper_parameters_space 
space_ = {
          'n_estimators': hp.choice('n_estimators', range(10,300,1)),
          'max_depth': hp.choice('max_depth', range(1,11,1)),
          'min_samples_split': hp.choice('min_samples_split', range(2,10,1)),
          'min_samples_leaf': hp.choice('min_samples_leaf', range(1,5,1)),
          'max_features': hp.uniform('max_features', 0.4, 1.0),
          'min_impurity_decrease': hp.uniform('min_impurity_decrease', 0.005,5),
          'criterion': hp.choice('criterion', ['gini', 'entropy']),
          'bootstrap': hp.choice('bootstrap', [True,False]),
#           'class_weight': hp.choice('class_weight', ['balanced', 'balanced_subsample',{}])
          }

n_estimators_ls = range(10,300,1)
max_depth_ls =  range(1,11,1)
min_samples_split_ls = range(2,10,1)
min_samples_leaf_ls = range(1,5,1)
criterion_ls = ['gini', 'entropy']
bootstrap_ls = [True,False]
# class_weight_ls = ['balanced', 'balanced_subsample',{}]

dataset = pd.read_csv(file_name)
pd_res = []

In [4]:
def hyper_runing(subtask):
    cols = [subtask]
    cols.extend(dataset.columns[(len(tasks) + 1):])
    sub_dataset = dataset[cols]

    # detect the na in the subtask (y cloumn)
    rm_index = sub_dataset[subtask][sub_dataset[subtask].isnull()].index
    sub_dataset.drop(index=rm_index, inplace=True)

    # remove the features with na
    sub_dataset = sub_dataset.dropna(axis=1)
    # *******************
    # demension reduction
    # *******************
    # Removing features with low variance
    # threshold = 0.05
    data_fea_var = sub_dataset.iloc[:, 2:].var()
    del_fea1 = list(data_fea_var[data_fea_var <= 0.05].index)
    sub_dataset.drop(columns=del_fea1, inplace=True)

    # pair correlations
    # threshold = 0.95
    data_fea_corr = sub_dataset.iloc[:, 2:].corr()
    del_fea2_col = []
    del_fea2_ind = []
    length = data_fea_corr.shape[1]
    for i in range(length):
        for j in range(i + 1, length):
            if abs(data_fea_corr.iloc[i, j]) >= 0.95:
                del_fea2_col.append(data_fea_corr.columns[i])
                del_fea2_ind.append(data_fea_corr.index[j])
    sub_dataset.drop(columns=del_fea2_ind, inplace=True)

    # standardize the features
    cols_ = list(sub_dataset.columns)[2:]
    if not ecfp :
        sub_dataset[cols_] = sub_dataset[cols_].apply(standardize, axis=0)

    # get the attentivefp data splits
    data_tr = sub_dataset[sub_dataset['group'] == 'train']
    data_va = sub_dataset[sub_dataset['group'] == 'valid']
    data_te = sub_dataset[sub_dataset['group'] == 'test']

    # prepare data for training
    # training set
    data_tr_y = data_tr[subtask].values.reshape(-1, 1)
    data_tr_x = np.array(data_tr.iloc[:, 2:].values)

    # validation set
    data_va_y = data_va[subtask].values.reshape(-1, 1)
    data_va_x = np.array(data_va.iloc[:, 2:].values)

    # test set
    data_te_y = data_te[subtask].values.reshape(-1, 1)
    data_te_x = np.array(data_te.iloc[:, 2:].values)

    if feature_selection:
        # univariate feature selection
        trans1 = SelectPercentile(f_classif, percentile=80)
        trans1.fit(data_tr_x, data_tr_y)
        data_tr_x = trans1.transform(data_tr_x)
        data_va_x = trans1.transform(data_va_x)
        data_te_x = trans1.transform(data_te_x)

        # select from model
        clf = XGBClassifier(random_state=1)
        clf = clf.fit(data_tr_x, data_tr_y)
        trans2 = SelectFromModel(clf, prefit=True)

        data_tr_x = trans2.transform(data_tr_x)
        data_va_x = trans2.transform(data_va_x)
        data_te_x = trans2.transform(data_te_x)

    num_fea = data_tr_x.shape[1]
    print('the num of retained features for the ' + dataset_label + ' ' + subtask + ' is:', num_fea)

    def hyper_opt(args):
        model = ExtraTreesClassifier(**args) if task_type == 'cla' else ExtraTreesRegressor(**args)

        model.fit(data_tr_x, data_tr_y)
        val_preds = model.predict_proba(data_va_x) if task_type == 'cla' else \
            model.predict(data_va_x)
        loss = 1 - roc_auc_score(data_va_y, val_preds[:, 1]) if task_type == 'cla' else np.sqrt(
            mean_squared_error(data_va_y, val_preds))
        return {'loss': loss, 'status': STATUS_OK}

    # start hyper-parameters optimization
    trials = Trials()
    best_results = fmin(hyper_opt, space_, algo=tpe.suggest, max_evals=OPT_ITERS, trials=trials, show_progressbar=False)
    print('the best hyper-parameters for ' + dataset_label + ' ' + subtask + ' are:  ', best_results)
    
    best_model = ExtraTreesClassifier(  
                                        n_estimators= n_estimators_ls[best_results['n_estimators']],
                                        max_depth = max_depth_ls[best_results['max_depth']],
                                        min_samples_split=min_samples_split_ls[best_results['min_samples_split']],
                                        min_samples_leaf=min_samples_leaf_ls[best_results['min_samples_leaf']],
                                        max_features=best_results['max_features'],
                                        min_impurity_decrease=best_results['min_impurity_decrease'],
                                        criterion=criterion_ls[best_results['criterion']],
                                        bootstrap=bootstrap_ls[best_results['bootstrap']],
#                                         class_weight=class_weight_ls[best_results['class_weight']]
                                      ) \
        if task_type == 'cla' else ExtraTreesRegressor(
                                        n_estimators= n_estimators_ls[best_results['n_estimators']],
                                        max_depth = max_depth_ls[best_results['max_depth']],
                                        min_samples_split=min_samples_split_ls[best_results['min_samples_split']],
                                        min_samples_leaf=min_samples_leaf_ls[best_results['min_samples_leaf']],
                                        max_features=best_results['max_features'],
                                        min_impurity_decrease=best_results['min_impurity_decrease'],
                                        criterion=criterion_ls[best_results['criterion']],
                                        bootstrap=bootstrap_ls[best_results['bootstrap']],
#                                         class_weight=class_weight_ls[best_results['class_weight']]
                                      )  
    
    best_model.fit(data_tr_x, data_tr_y)
    
    num_of_compounds = len(sub_dataset)

    if task_type == 'cla':
        # training set
        tr_pred = best_model.predict_proba(data_tr_x)
        tr_results = [dataset_label, subtask, 'tr', num_fea, num_of_compounds, data_tr_y[data_tr_y == 1].shape[0],
                      data_tr_y[data_tr_y == 0].shape[0],
                      data_tr_y[data_tr_y == 0].shape[0] / data_tr_y[data_tr_y == 1].shape[0],
                      n_estimators_ls[best_results['n_estimators']],
                      max_depth_ls[best_results['max_depth']],
                      min_samples_split_ls[best_results['min_samples_split']],
                      min_samples_leaf_ls[best_results['min_samples_leaf']],
                      best_results['max_features'],
                      best_results['min_impurity_decrease'],
                      criterion_ls[best_results['criterion']],
                      bootstrap_ls[best_results['bootstrap']],
#                       class_weight_ls[best_results['class_weight']]
                     ]
        tr_results.extend(statistical(data_tr_y, np.argmax(tr_pred, axis=1), tr_pred[:, 1]))
        # validation set
        va_pred = best_model.predict_proba(data_va_x)
                      
        va_results = [dataset_label, subtask, 'va', num_fea, num_of_compounds, data_va_y[data_va_y == 1].shape[0],
                      data_va_y[data_va_y == 0].shape[0],
                      data_va_y[data_va_y == 0].shape[0] / data_va_y[data_va_y == 1].shape[0],
                      n_estimators_ls[best_results['n_estimators']],
                      max_depth_ls[best_results['max_depth']],
                      min_samples_split_ls[best_results['min_samples_split']],
                      min_samples_leaf_ls[best_results['min_samples_leaf']],
                      best_results['max_features'],
                      best_results['min_impurity_decrease'],
                      criterion_ls[best_results['criterion']],
                      bootstrap_ls[best_results['bootstrap']],
#                       class_weight_ls[best_results['class_weight']]
                     ]
        va_results.extend(statistical(data_va_y, np.argmax(va_pred, axis=1), va_pred[:, 1]))

        # test set
        te_pred = best_model.predict_proba(data_te_x)
        te_results = [dataset_label, subtask, 'te', num_fea, num_of_compounds, data_te_y[data_te_y == 1].shape[0],
                      data_te_y[data_te_y == 0].shape[0],
                      data_te_y[data_te_y == 0].shape[0] / data_te_y[data_te_y == 1].shape[0],
                      n_estimators_ls[best_results['n_estimators']],
                      max_depth_ls[best_results['max_depth']],
                      min_samples_split_ls[best_results['min_samples_split']],
                      min_samples_leaf_ls[best_results['min_samples_leaf']],
                      best_results['max_features'],
                      best_results['min_impurity_decrease'],
                      criterion_ls[best_results['criterion']],
                      bootstrap_ls[best_results['bootstrap']],
#                       class_weight_ls[best_results['class_weight']]
                     ]
        te_results.extend(statistical(data_te_y, np.argmax(te_pred, axis=1), te_pred[:, 1]))
    else:
        # training set
        tr_pred = best_model.predict(data_tr_x)
        tr_results = [dataset_label, subtask, 'tr', num_fea, num_of_compounds,
                      n_estimators_ls[best_results['n_estimators']],
                      max_depth_ls[best_results['max_depth']],
                      min_samples_split_ls[best_results['min_samples_split']],
                      min_samples_leaf_ls[best_results['min_samples_leaf']],
                      best_results['max_features'],
                      best_results['min_impurity_decrease'],
                      criterion_ls[best_results['criterion']],
                      bootstrap_ls[best_results['bootstrap']],
#                       class_weight_ls[best_results['class_weight']],
                      np.sqrt(mean_squared_error(data_tr_y, tr_pred)), r2_score(data_tr_y, tr_pred),
                      mean_absolute_error(data_tr_y, tr_pred)]

        # validation set
        va_pred = best_model.predict(data_va_x)
        va_results = [dataset_label, subtask, 'va', num_fea, num_of_compounds,
                      n_estimators_ls[best_results['n_estimators']],
                      max_depth_ls[best_results['max_depth']],
                      min_samples_split_ls[best_results['min_samples_split']],
                      min_samples_leaf_ls[best_results['min_samples_leaf']],
                      best_results['max_features'],
                      best_results['min_impurity_decrease'],
                      criterion_ls[best_results['criterion']],
                      bootstrap_ls[best_results['bootstrap']],
#                       class_weight_ls[best_results['class_weight']],
                      np.sqrt(mean_squared_error(data_va_y, va_pred)), r2_score(data_va_y, va_pred),
                      mean_absolute_error(data_va_y, va_pred)]

        # test set
        te_pred = best_model.predict(data_te_x)
        te_results = [dataset_label, subtask, 'te', num_fea, num_of_compounds,
                      n_estimators_ls[best_results['n_estimators']],
                      max_depth_ls[best_results['max_depth']],
                      min_samples_split_ls[best_results['min_samples_split']],
                      min_samples_leaf_ls[best_results['min_samples_leaf']],
                      best_results['max_features'],
                      best_results['min_impurity_decrease'],
                      criterion_ls[best_results['criterion']],
                      bootstrap_ls[best_results['bootstrap']],
#                       class_weight_ls[best_results['class_weight']],
                      np.sqrt(mean_squared_error(data_te_y, te_pred)), r2_score(data_te_y, te_pred),
                      mean_absolute_error(data_te_y, te_pred)]
    return tr_results, va_results, te_results


pool = multiprocessing.Pool(num_pools)
res = pool.starmap(hyper_runing, zip(tasks))
pool.close()
pool.join()
for item in res:
    for i in range(3):
        pd_res.append(item[i])
if task_type == 'cla':       
                                            
    best_hyper = pd.DataFrame(pd_res, columns=['dataset', 'subtask', 'set',
                                               'num_of_retained_feature',
                                               'num_of_compounds', 'postives',
                                               'negtives', 'negtives/postives',
                                               'n_estimators',
                                               'max_depth',
                                               'min_samples_split','min_samples_leaf','max_features','min_impurity_decrease',
                                               'criterion','bootstrap',
#                                                'class_weight',
                                               'tn', 'fp', 'fn', 'tp', 'se', 'sp',
                                               'auc_prc', 'acc', 'auc_roc','recall','precision','f1','kappa','mcc'])
else:
    best_hyper = pd.DataFrame(pd_res, columns=['dataset', 'subtask', 'set',
                                               'n_estimators',
                                               'max_depth',
                                               'min_samples_split','min_samples_leaf','max_features','min_impurity_decrease',
                                               'criterion','bootstrap',
#                                                'class_weight',
                                               'rmse', 'r2', 'mae'])
best_hyper.to_csv('./model/' + dataset_label + '_EXT_hyperopt_info.csv', index=0)

if task_type == 'cla':
    print('train', best_hyper[best_hyper['set'] == 'tr']['auc_roc'].mean(),
          best_hyper[best_hyper['set'] == 'tr']['auc_prc'].mean())
    print('valid', best_hyper[best_hyper['set'] == 'va']['auc_roc'].mean(),
          best_hyper[best_hyper['set'] == 'va']['auc_prc'].mean())
    print('test', best_hyper[best_hyper['set'] == 'te']['auc_roc'].mean(),
          best_hyper[best_hyper['set'] == 'te']['auc_prc'].mean())
else:
    print('train', best_hyper[best_hyper['set'] == 'tr']['rmse'].mean(),
          best_hyper[best_hyper['set'] == 'tr']['r2'].mean(), best_hyper[best_hyper['set'] == 'tr']['mae'].mean())
    print('valid', best_hyper[best_hyper['set'] == 'va']['rmse'].mean(),
          best_hyper[best_hyper['set'] == 'va']['r2'].mean(), best_hyper[best_hyper['set'] == 'va']['mae'].mean())
    print('test', best_hyper[best_hyper['set'] == 'te']['rmse'].mean(),
          best_hyper[best_hyper['set'] == 'te']['r2'].mean(), best_hyper[best_hyper['set'] == 'te']['mae'].mean())
    

the num of retained features for the 1-AR-Alva-6108-slim-Normalize-group.csv activity is: 1508
the best hyper-parameters for 1-AR-Alva-6108-slim-Normalize-group.csv activity are:   {'bootstrap': 0, 'criterion': 1, 'max_depth': 9, 'max_features': 0.6558709526720025, 'min_impurity_decrease': 0.06106556710136113, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 216}
train 0.8829790786993965 0.7209547671283796
valid 0.8698662386302835 0.6696306159114056
test 0.8902268224362916 0.7127823376836466


In [5]:
# 10 repetitions based on thr best hypers
dataset.drop(columns=['group'], inplace=True)

In [6]:
pd_res = []
def best_model_runing(split):
    seed = split
    if task_type == 'cla':
        while True:
            training_data, data_te = train_test_split(sub_dataset, test_size=0.1, random_state=seed)
            # the training set was further splited into the training set and validation set
            data_tr, data_va = train_test_split(training_data, test_size=0.1, random_state=seed)
            if (all_one_zeros(data_tr[subtask]) or all_one_zeros(data_va[subtask]) or all_one_zeros(data_te[subtask])):
                print(
                    '\ninvalid random seed {} due to one class presented in the {} splitted sets...'.format(seed,
                                                                                                            subtask))
                print('Changing to another random seed...\n')
                seed = np.random.randint(10, 999999)
            else:
                print('random seed used in repetition {} is {}'.format(split, seed))
                break
    else:
        training_data, data_te = train_test_split(sub_dataset, test_size=0.1, random_state=seed)
        # the training set was further splited into the training set and validation set
        data_tr, data_va = train_test_split(training_data, test_size=0.1, random_state=seed)

    # prepare data for training
    # training set
    data_tr_y = data_tr[subtask].values.reshape(-1, 1)
    data_tr_x = np.array(data_tr.iloc[:, 1:].values)

    # validation set
    data_va_y = data_va[subtask].values.reshape(-1, 1)
    data_va_x = np.array(data_va.iloc[:, 1:].values)

    # test set
    data_te_y = data_te[subtask].values.reshape(-1, 1)
    data_te_x = np.array(data_te.iloc[:, 1:].values)

    if feature_selection:
        # univariate feature selection
        trans1 = SelectPercentile(f_classif, percentile=80)
        trans1.fit(data_tr_x, data_tr_y)
        data_tr_x = trans1.transform(data_tr_x)
        data_va_x = trans1.transform(data_va_x)
        data_te_x = trans1.transform(data_te_x)

        # select from model
        clf = XGBClassifier(random_state=1)
        clf = clf.fit(data_tr_x, data_tr_y)
        trans2 = SelectFromModel(clf, prefit=True)

        data_tr_x = trans2.transform(data_tr_x)
        data_va_x = trans2.transform(data_va_x)
        data_te_x = trans2.transform(data_te_x)     
        
    num_fea = data_tr_x.shape[1]
    pos_weight = (len(sub_dataset) - sum(sub_dataset[subtask])) / sum(sub_dataset[subtask])
    model = ExtraTreesClassifier(
                          n_estimators=best_hyper[best_hyper.subtask == subtask].iloc[0,]['n_estimators'],
                          max_depth=best_hyper[best_hyper.subtask == subtask].iloc[0,]['max_depth'],
                          min_samples_split=best_hyper[best_hyper.subtask == subtask].iloc[0,]['min_samples_split'],
                          min_samples_leaf=best_hyper[best_hyper.subtask == subtask].iloc[0,]['min_samples_leaf'],
                          max_features=best_hyper[best_hyper.subtask == subtask].iloc[0,]['max_features'],
                          min_impurity_decrease=best_hyper[best_hyper.subtask == subtask].iloc[0,]['min_impurity_decrease'],
                          criterion=best_hyper[best_hyper.subtask == subtask].iloc[0,]['criterion'],
                          bootstrap=best_hyper[best_hyper.subtask == subtask].iloc[0,]['bootstrap'],
#                           class_weight=best_hyper[best_hyper.subtask == subtask].iloc[0,]['class_weight']
                          ) \
        if task_type == 'cla' else ExtraTreesRegressor(
                          n_estimators=best_hyper[best_hyper.subtask == subtask].iloc[0,]['n_estimators'],
                          max_depth=best_hyper[best_hyper.subtask == subtask].iloc[0,]['max_depth'],
                          min_samples_split=best_hyper[best_hyper.subtask == subtask].iloc[0,]['min_samples_split'],
                          min_samples_leaf=best_hyper[best_hyper.subtask == subtask].iloc[0,]['min_samples_leaf'],
                          max_features=best_hyper[best_hyper.subtask == subtask].iloc[0,]['max_features'],
                          min_impurity_decrease=best_hyper[best_hyper.subtask == subtask].iloc[0,]['min_impurity_decrease'],
                          criterion=best_hyper[best_hyper.subtask == subtask].iloc[0,]['criterion'],
                          bootstrap=best_hyper[best_hyper.subtask == subtask].iloc[0,]['bootstrap'],
#                           class_weight=best_hyper[best_hyper.subtask == subtask].iloc[0,]['class_weight']
                          )

    model.fit(data_tr_x, data_tr_y)
    num_of_compounds = sub_dataset.shape[0]
    import pickle
    pickle.dump(model, open("./model/ext_"+str(split)+".pkl", "wb"))
    if task_type == 'cla':
        # training set
        tr_pred = model.predict_proba(data_tr_x)
        tr_results = [split, dataset_label, subtask, 'tr', num_fea, num_of_compounds,
                      data_tr_y[data_tr_y == 1].shape[0],
                      data_tr_y[data_tr_y == 0].shape[0],
                      data_tr_y[data_tr_y == 0].shape[0] / data_tr_y[data_tr_y == 1].shape[0]]
        tr_results.extend(statistical(data_tr_y, np.argmax(tr_pred, axis=1), tr_pred[:, 1]))

        # validation set
        va_pred = model.predict_proba(data_va_x)
        va_results = [split, dataset_label, subtask, 'va', num_fea, num_of_compounds,
                      data_va_y[data_va_y == 1].shape[0],
                      data_va_y[data_va_y == 0].shape[0],
                      data_va_y[data_va_y == 0].shape[0] / data_va_y[data_va_y == 1].shape[0]]
        va_results.extend(statistical(data_va_y, np.argmax(va_pred, axis=1), va_pred[:, 1]))

        # test set
        te_pred = model.predict_proba(data_te_x)
        te_results = [split, dataset_label, subtask, 'te', num_fea, num_of_compounds,
                      data_te_y[data_te_y == 1].shape[0],
                      data_te_y[data_te_y == 0].shape[0],
                      data_te_y[data_te_y == 0].shape[0] / data_te_y[data_te_y == 1].shape[0]]
        te_results.extend(statistical(data_te_y, np.argmax(te_pred, axis=1), te_pred[:, 1]))
    else:
        # training set
        tr_pred = model.predict(data_tr_x)
        tr_results = [split, dataset_label, subtask, 'tr', num_fea, num_of_compounds,
                      np.sqrt(mean_squared_error(data_tr_y, tr_pred)), r2_score(data_tr_y, tr_pred),
                      mean_absolute_error(data_tr_y, tr_pred)]

        # validation set
        va_pred = model.predict(data_va_x)
        va_results = [split, dataset_label, subtask, 'va', num_fea, num_of_compounds,
                      np.sqrt(mean_squared_error(data_va_y, va_pred)), r2_score(data_va_y, va_pred),
                      mean_absolute_error(data_va_y, va_pred)]

        # test set
        te_pred = model.predict(data_te_x)
        te_results = [split, dataset_label, subtask, 'te', num_fea, num_of_compounds,
                      np.sqrt(mean_squared_error(data_te_y, te_pred)), r2_score(data_te_y, te_pred),
                      mean_absolute_error(data_te_y, te_pred)]
    return tr_results, va_results, te_results


for subtask in tasks:
    cols = [subtask]
    cols.extend(dataset.columns[(len(tasks) + 1):])
    sub_dataset = dataset[cols]

    # detect the NA in the subtask (y cloumn)
    rm_index = sub_dataset[subtask][sub_dataset[subtask].isnull()].index
    sub_dataset.drop(index=rm_index, inplace=True)

    # remove the features with na
    sub_dataset = sub_dataset.dropna(axis=1)
    # *******************
    # demension reduction
    # *******************
    # Removing features with low variance
    # threshold = 0.05
    data_fea_var = sub_dataset.iloc[:, 1:].var()
    del_fea1 = list(data_fea_var[data_fea_var <= 0.05].index)
    sub_dataset.drop(columns=del_fea1, inplace=True)

    # pair correlations
    # threshold = 0.95
    data_fea_corr = sub_dataset.iloc[:, 1:].corr()
    del_fea2_col = []
    del_fea2_ind = []
    length = data_fea_corr.shape[1]
    for i in range(length):
        for j in range(i + 1, length):
            if abs(data_fea_corr.iloc[i, j]) >= 0.95:
                del_fea2_col.append(data_fea_corr.columns[i])
                del_fea2_ind.append(data_fea_corr.index[j])
    sub_dataset.drop(columns=del_fea2_ind, inplace=True)

    # standardize the features
    cols_ = list(sub_dataset.columns)[1:]
    if not ecfp :
        sub_dataset[cols_] = sub_dataset[cols_].apply(standardize, axis=0)

    # for split in range(1, splits+1):
    pool = multiprocessing.Pool(num_pools)
    res = pool.starmap(best_model_runing, zip(range(1, repetitions + 1)))
    pool.close()
    pool.join()
    for item in res:
        for i in range(3):
            pd_res.append(item[i])
if task_type == 'cla':
    stat_res = pd.DataFrame(pd_res, columns=['split', 'dataset', 'subtask', 'set',
                                             'num_of_retained_feature',
                                             'num_of_compounds', 'postives',
                                             'negtives', 'negtives/postives',
                                             'tn', 'fp', 'fn', 'tp', 'se', 'sp',
                                             'auc_prc', 'acc', 'auc_roc','recall','precision','f1','kappa','mcc'])
else:
    stat_res = pd.DataFrame(pd_res, columns=['split', 'dataset', 'subtask', 'set',
                                             'num_of_retained_feature',
                                             'num_of_compounds', 'rmse', 'r2', 'mae'])
stat_res.to_csv('./model/' + dataset_label + '_EXT_statistical_results_split.csv', index=0)
# single tasks
if len(tasks) == 1:
    args = {'data_label': dataset_label, 'metric': 'auc_roc' if task_type == 'cla' else 'rmse', 'model': 'EXT'}
    print('{}_{}: the mean {} for the training set is {:.3f} with std {:.3f}'.format(args['data_label'], args['model'],
                                                                                     args['metric'], np.mean(
            stat_res[stat_res['set'] == 'tr'][args['metric']]), np.std(
            stat_res[stat_res['set'] == 'tr'][args['metric']])))
    print(
        '{}_{}: the mean {} for the validation set is {:.3f} with std {:.3f}'.format(args['data_label'], args['model'],
                                                                                     args['metric'], np.mean(
                stat_res[stat_res['set'] == 'va'][args['metric']]), np.std(
                stat_res[stat_res['set'] == 'va'][args['metric']])))
    print('{}_{}: the mean {} for the test set is {:.3f} with std {:.3f}'.format(args['data_label'], args['model'],
                                                                                 args['metric'], np.mean(
            stat_res[stat_res['set'] == 'te'][args['metric']]), np.std(
            stat_res[stat_res['set'] == 'te'][args['metric']])))
# multi-tasks
else:
    args = {'data_label': dataset_label, 'metric': 'auc_roc' if dataset_label != 'muv' else 'auc_prc', 'model': 'EXT'}
    tr_acc = np.zeros(repetitions)
    va_acc = np.zeros(repetitions)
    te_acc = np.zeros(repetitions)
    for subtask in tasks:
        tr = stat_res[stat_res['set'] == 'tr']
        tr_acc = tr_acc + tr[tr['subtask'] == subtask][args['metric']].values

        va = stat_res[stat_res['set'] == 'va']
        va_acc = va_acc + va[va['subtask'] == subtask][args['metric']].values

        te = stat_res[stat_res['set'] == 'te']
        te_acc = te_acc + te[te['subtask'] == subtask][args['metric']].values
    tr_acc = tr_acc / len(tasks)
    va_acc = va_acc / len(tasks)
    te_acc = te_acc / len(tasks)
    print('{}_{}: the mean {} for the training set is {:.3f} with std {:.3f}'.format(args['data_label'], args['model'],
                                                                                     args['metric'], np.mean(tr_acc),
                                                                                     np.std(tr_acc)))
    print(
        '{}_{}: the mean {} for the validation set is {:.3f} with std {:.3f}'.format(args['data_label'], args['model'],
                                                                                     args['metric'], np.mean(va_acc),
                                                                                     np.std(va_acc)))
    print('{}_{}: the mean {} for the test set is {:.3f} with std {:.3f}'.format(args['data_label'], args['model'],
                                                                                 args['metric'], np.mean(te_acc),
                                                                                 np.std(te_acc)))
end = time.time()  # get the end time

random seed used in repetition 2 is 2
random seed used in repetition 6 is 6
random seed used in repetition 5 is 5
random seed used in repetition 1 is 1
random seed used in repetition 8 is 8
random seed used in repetition 4 is 4
random seed used in repetition 10 is 10
random seed used in repetition 3 is 3
random seed used in repetition 7 is 7
random seed used in repetition 9 is 9
1-AR-Alva-6108-slim-Normalize-group.csv_EXT: the mean auc_roc for the training set is 0.883 with std 0.002
1-AR-Alva-6108-slim-Normalize-group.csv_EXT: the mean auc_roc for the validation set is 0.884 with std 0.012
1-AR-Alva-6108-slim-Normalize-group.csv_EXT: the mean auc_roc for the test set is 0.887 with std 0.014


In [7]:
# acc auc_roc recall precision f1 kappa mcc
acc_str = 'acc of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['acc']), 
                np.std(stat_res[stat_res['set'] == 'tr']['acc']),
                np.mean(stat_res[stat_res['set'] == 'va']['acc']), 
                np.std(stat_res[stat_res['set'] == 'va']['acc']),
                np.mean(stat_res[stat_res['set'] == 'te']['acc']), 
                np.std(stat_res[stat_res['set'] == 'te']['acc']),
)
auc_str = 'auc_roc of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['auc_roc']), 
                np.std(stat_res[stat_res['set'] == 'tr']['auc_roc']),
                np.mean(stat_res[stat_res['set'] == 'va']['auc_roc']), 
                np.std(stat_res[stat_res['set'] == 'va']['auc_roc']),
                np.mean(stat_res[stat_res['set'] == 'te']['auc_roc']), 
                np.std(stat_res[stat_res['set'] == 'te']['auc_roc']),
)
recall_str = 'recall of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['recall']), 
                np.std(stat_res[stat_res['set'] == 'tr']['recall']),
                np.mean(stat_res[stat_res['set'] == 'va']['recall']), 
                np.std(stat_res[stat_res['set'] == 'va']['recall']),
                np.mean(stat_res[stat_res['set'] == 'te']['recall']), 
                np.std(stat_res[stat_res['set'] == 'te']['recall']),
)
precision_str = 'precision of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['precision']), 
                np.std(stat_res[stat_res['set'] == 'tr']['precision']),
                np.mean(stat_res[stat_res['set'] == 'va']['precision']), 
                np.std(stat_res[stat_res['set'] == 'va']['precision']),
                np.mean(stat_res[stat_res['set'] == 'te']['precision']), 
                np.std(stat_res[stat_res['set'] == 'te']['precision']),
)
f1_str = 'f1 of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['f1']), 
                np.std(stat_res[stat_res['set'] == 'tr']['f1']),
                np.mean(stat_res[stat_res['set'] == 'va']['f1']), 
                np.std(stat_res[stat_res['set'] == 'va']['f1']),
                np.mean(stat_res[stat_res['set'] == 'te']['f1']), 
                np.std(stat_res[stat_res['set'] == 'te']['f1']),
)
kappa_str = 'kappa of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['kappa']), 
                np.std(stat_res[stat_res['set'] == 'tr']['kappa']),
                np.mean(stat_res[stat_res['set'] == 'va']['kappa']), 
                np.std(stat_res[stat_res['set'] == 'va']['kappa']),
                np.mean(stat_res[stat_res['set'] == 'te']['kappa']), 
                np.std(stat_res[stat_res['set'] == 'te']['kappa']),
)
mcc_str = 'mcc of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['mcc']), 
                np.std(stat_res[stat_res['set'] == 'tr']['mcc']),
                np.mean(stat_res[stat_res['set'] == 'va']['mcc']), 
                np.std(stat_res[stat_res['set'] == 'va']['mcc']),
                np.mean(stat_res[stat_res['set'] == 'te']['mcc']), 
                np.std(stat_res[stat_res['set'] == 'te']['mcc']),
)
print('the elapsed time is:', (end - start)/3600, 'H')

the elapsed time is: 0.13758470992247263 H


In [8]:
print(acc_str)
print(auc_str)
print(recall_str)
print(precision_str)
print(f1_str)
print(kappa_str)
print(mcc_str)

acc of training set is 0.855±0.008, validation set is 0.847±0.016, test set is 0.850±0.016
auc_roc of training set is 0.883±0.002, validation set is 0.884±0.012, test set is 0.887±0.014
recall of training set is 0.245±0.045, validation set is 0.222±0.053, test set is 0.246±0.041
precision of training set is 0.964±0.017, validation set is 0.952±0.046, test set is 0.948±0.038
f1 of training set is 0.389±0.061, validation set is 0.355±0.076, test set is 0.389±0.052
kappa of training set is 0.339±0.056, validation set is 0.306±0.067, test set is 0.336±0.050
mcc of training set is 0.442±0.044, validation set is 0.412±0.057, test set is 0.437±0.038


In [9]:
with open('output/output_ext.txt', 'w') as f:
    f.write(acc_str+'\n')
    f.write(auc_str+'\n')
    f.write(recall_str+'\n')
    f.write(precision_str+'\n')
    f.write(f1_str+'\n')
    f.write(kappa_str+'\n')
    f.write(mcc_str+'\n')
    f.write(str(cols_)+'\n')
cols_ = pd.DataFrame(cols_)
cols_.to_csv('output/output_ext_cols.csv',index=False)

In [10]:
import pandas as pd
import collections
dict1 = {"model: EXT ":['acc','auc_roc','recall','precision','f1','kappa','mcc','auc_prc'],
         "Train":[np.mean(stat_res[stat_res['set'] == 'tr']['acc']),np.mean(stat_res[stat_res['set'] == 'tr']['auc_roc']),
                  np.mean(stat_res[stat_res['set'] == 'tr']['recall']),np.mean(stat_res[stat_res['set'] == 'tr']['precision']),
                  np.mean(stat_res[stat_res['set'] == 'tr']['f1']),np.mean(stat_res[stat_res['set'] == 'tr']['kappa']),
                  np.mean(stat_res[stat_res['set'] == 'tr']['mcc']),np.mean(stat_res[stat_res['set'] == 'tr']['auc_prc']),                                     
                 ],
         "Tr_STD":[np.std(stat_res[stat_res['set'] == 'tr']['acc']),np.std(stat_res[stat_res['set'] == 'tr']['auc_roc']),
                  np.std(stat_res[stat_res['set'] == 'tr']['recall']),np.std(stat_res[stat_res['set'] == 'tr']['precision']),
                  np.std(stat_res[stat_res['set'] == 'tr']['f1']),np.std(stat_res[stat_res['set'] == 'tr']['kappa']),
                  np.std(stat_res[stat_res['set'] == 'tr']['mcc']),np.std(stat_res[stat_res['set'] == 'tr']['auc_prc']),],
         "Validation":[np.mean(stat_res[stat_res['set'] == 'va']['acc']),np.mean(stat_res[stat_res['set'] == 'va']['auc_roc']),
                      np.mean(stat_res[stat_res['set'] == 'va']['recall']),np.mean(stat_res[stat_res['set'] == 'va']['precision']),
                      np.mean(stat_res[stat_res['set'] == 'va']['f1']),np.mean(stat_res[stat_res['set'] == 'va']['kappa']),
                      np.mean(stat_res[stat_res['set'] == 'va']['mcc']),np.mean(stat_res[stat_res['set'] == 'va']['auc_prc'])],
         "Va_STD":[np.std(stat_res[stat_res['set'] == 'va']['acc']),np.std(stat_res[stat_res['set'] == 'va']['auc_roc']),
                  np.std(stat_res[stat_res['set'] == 'va']['recall']),np.std(stat_res[stat_res['set'] == 'va']['precision']),
                  np.std(stat_res[stat_res['set'] == 'va']['f1']),np.std(stat_res[stat_res['set'] == 'va']['kappa']),
                  np.std(stat_res[stat_res['set'] == 'va']['mcc']),np.std(stat_res[stat_res['set'] == 'va']['auc_prc'])],
         "Test":[np.mean(stat_res[stat_res['set'] == 'te']['acc']),np.mean(stat_res[stat_res['set'] == 'te']['auc_roc']),
                np.mean(stat_res[stat_res['set'] == 'te']['recall']),np.mean(stat_res[stat_res['set'] == 'te']['precision']),
                np.mean(stat_res[stat_res['set'] == 'te']['f1']),np.mean(stat_res[stat_res['set'] == 'te']['kappa']),
                np.mean(stat_res[stat_res['set'] == 'te']['mcc']),np.mean(stat_res[stat_res['set'] == 'te']['auc_prc'])],
          "Te_STD":[np.std(stat_res[stat_res['set'] == 'te']['acc']),np.std(stat_res[stat_res['set'] == 'te']['auc_roc']),
                   np.std(stat_res[stat_res['set'] == 'te']['recall']),np.std(stat_res[stat_res['set'] == 'te']['precision']),
                   np.std(stat_res[stat_res['set'] == 'te']['f1']),np.std(stat_res[stat_res['set'] == 'te']['kappa']),
                   np.std(stat_res[stat_res['set'] == 'te']['mcc']),np.std(stat_res[stat_res['set'] == 'te']['auc_prc']),]}
dict1 = collections.OrderedDict(dict1)
df = pd.DataFrame(dict1,index = None)
df.to_csv('output/output_ext.csv',index = False)
df

,model: EXT,Train,Tr_STD,Validation,Va_STD,Test,Te_STD
0,acc,0.854720,0.007594,0.846545,0.016368,0.850082,0.015753
1,auc_roc,0.883264,0.002108,0.883563,0.012418,0.887160,0.013718
2,recall,0.245474,0.045124,0.221587,0.053045,0.246461,0.041425
3,precision,0.963909,0.017294,0.952471,0.046105,0.948153,0.038159
4,f1,0.388627,0.061462,0.355449,0.076071,0.388821,0.052466
5,kappa,0.338508,0.055574,0.305953,0.067380,0.336431,0.050253
6,mcc,0.442097,0.043776,0.412084,0.057369,0.437277,0.037783
7,auc_prc,0.717208,0.006708,0.711516,0.022814,0.720198,0.031138


In [11]:
pd.set_option ( 'display.width', None)
pd.set_option ( 'display.max_columns', None) #显示全部列
hyper_parameters = best_hyper.iloc[0:1,8:-14].T
hyper_parameters.rename(columns={0:'Values'},inplace=True) 
hyper_parameters

,Values
n_estimators,226
max_depth,10
min_samples_split,4
min_samples_leaf,2
max_features,0.655871
min_impurity_decrease,0.0610656
criterion,entropy
bootstrap,True
